In [ ]:
# library information:

import os
# It is the fundamental library of python, used to perform scientific computing. 
# It provides high-performance multidimensional arrays and tools to deal with them.
import numpy as np
# Pandas provide high performance, fast, easy to use data structures and data analysis tools for manipulating 
# numeric data and time series. Pandas is built on the numpy library and written in languages like Python.
import pandas as pd
# ------------------------
import sys
sys.path.append('Saeed/functions/')
sys.path.append('Saeed/functions/utils/')
# and now import my functions
from my_functions import get_tickers, create_dataframe, get_yf_data
from date_functions import get_begin_date, get_present_time
from test_sr import display_greeting
# ------------------------
# import library to read date range for stock ticker historical data range
import datetime
# import json
# from pathlib import Path
# import alpaca_trade_api as tradeapi
# from dotenv import load_dotenv
# ------------------------
# Use this api to get data.
import yfinance as yf
# ------------------------
%matplotlib inline

In [ ]:
# Test folders: Do I have access?
display_greeting("Saeed")

In [ ]:
# INITIAL: Figure out number of days needed to pull data for

# number of days for which we need data. for 200SMA we should atleast get 250 days
num_of_days = 365    # 365 - Get atleast a year's worth of data.
num_of_days

In [ ]:
# INITIAL: Get End Time & Start Time, String & ISO format

# Decipher Time - End Time  ... String & ISO

# format current date as ISO format
this_moment = datetime.datetime.now()
end_date_str = str(this_moment).split()[0]
end_date_iso = pd.Timestamp(end_date_str, tz="America/New_York").isoformat()

# Decipher Time - Begin Time  ... String & ISO

strt_dte = get_begin_date(this_moment, num_of_days)
start_date_str = str(strt_dte).split()[0]
start_date_iso = pd.Timestamp(start_date_str, tz="America/New_York").isoformat()

print(end_date_str)
print(end_date_iso)
print(start_date_str)
print(start_date_iso)

In [ ]:
# INITIAL: Get Stock Market Tickers

# Dow - has 30 symbols
# tickers_dow = get_tickers("dow")     #     - 5 indicators: 2 mins 12.4 seconds
# choice = "dow"

# # sp500 - has 505 symbols
# tickers_sp500 = get_tickers("sp500")
# choice = "sp500"

# nasdaq has 4942 symbols
# tickers_nasdaq = get_tickers("nasdaq")
# choice = "nasdaq"
# print(tickers_nasdaq)

# Sams API Call for sp500
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
choice = "sp500"
sp500_tickers = sp500_tickers.Symbol.to_list()
# SAM: Clean Tickers of periods
sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
# print(sp500_tickers)

all_tickers = []

#all_tickers = tickers_dow
#all_tickers = tickers_sp500     #      Me
all_tickers = sp500_tickers     #      Sams's Code
#all_tickers = tickers_nasdaq

# Test Sample
# all_tickers = ["WMT", "BBY", "BAX", "BBWI", "AAPL"]     # ["AAPL", "V", "WMT"]
# choice = "sample"

all_tickers.sort()
print(all_tickers)

In [ ]:
# sp500: 70 seconds to 80 seconds
start_time = get_present_time()
print("..... Building Indicators: Please Wait .....")
print()
print()
print("Start Time: .......... ", start_time)
print()
selection = get_yf_data(all_tickers, "1y", "tickers")
print()
end_time = get_present_time()
print("End Time: ............ ", end_time)
selection

In [ ]:

start_time = get_present_time()
print("..... Building Indicators: Please Wait .....")
print()
print()
print("Start Time: .......... ", start_time)
print()

current_price = {}
dept_to_equity = {}
total_debt = {}
total_revenue = {}
market_cap = {}
forwardPE = {}

count = 1
for ticker in all_tickers:
    
    try:
        # ?hrs : ?mins : ?seconds
        if ticker != 'BBWI':
            info_all = yf.Ticker(ticker).info
    except Exception as e:
        print("Handling Ticker: '" + ticker + "'. The value of ERROR is: " + str(e))
        info_cp = None
        info_dte = None
        info_td = None
        info_tr = None
        info_mc = None
        info_fpe = None
        
    if ticker != 'BBWI':
        current_price[ticker] = info_all['currentPrice']
        dept_to_equity[ticker] = info_all['debtToEquity']
        total_debt[ticker] = info_all['totalDebt']
        total_revenue[ticker] = info_all['totalRevenue']
        market_cap[ticker] = info_all['marketCap']
        forwardPE[ticker] = info_all['forwardPE']    
        print("count: " + str(count) + "  -  ticker: " + ticker + "     - currentPrice: " + str(info_all['currentPrice']) + " - debtToEquity: " + str(info_all['debtToEquity']) + " - totalDebt: " + str(info_all['totalDebt']) + " - totalRevenue: " + str(info_all['totalRevenue']) + " - marketCap: " + str(info_all['marketCap']) + " - forwardPE: " + str(info_all['forwardPE']))
        count+=1
    
df_cp = pd.DataFrame(list(current_price.items()), columns=["ticker", "Current Price"])
df_dte = pd.DataFrame(list(dept_to_equity.items()), columns=["ticker", "Debt To Equity"])
df_td = pd.DataFrame(list(total_debt.items()), columns=["ticker", "Total Debt"])
df_tr = pd.DataFrame(list(total_revenue.items()), columns=["ticker", "Total Revenue"])
df_mc = pd.DataFrame(list(market_cap.items()), columns=["ticker", "Market Cap"])
df_fpe = pd.DataFrame(list(forwardPE.items()), columns=["ticker", "forwardPE"])

df_cp.set_index('ticker')
df_dte.set_index('ticker')
df_td.set_index('ticker')
df_tr.set_index('ticker')
df_mc.set_index('ticker')
df_fpe.set_index('ticker')

df_current_price_sorted = df_cp.sort_values(['Current Price'])
df_debt_to_equity_sorted = df_dte.sort_values('Debt To Equity')
df_total_debt_sorted = df_td.sort_values("Total Debt")
df_total_revenue_sorted = df_tr.sort_values("Total Revenue", ascending=[0])
df_market_cap_sorted = df_mc.sort_values("Market Cap", ascending=[0])
df_forward_PE__sorted = df_fpe.sort_values("forwardPE")


print()
end_time = get_present_time()
print("End Time: ............ ", end_time)


In [ ]:
df_market_cap_sorted